In [8]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd
import json
import fiona
import geopandas as gp
import jsonlines
es = Elasticsearch(['https://3d6a9dd50c7c49c9ab5d23b6891bc03e.us-central1.gcp.cloud.es.io:9243'], 
                    http_auth=('elastic', 'WMzYk5RXyzE7MRShwPVwHzPX'), timeout=30)
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 13.3 ms (started: 2022-01-11 09:24:55 -05:00)


In [2]:
STATE_FIPS_DICT_52 = { '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', 
                      '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', 
                      '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', 
                      '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', 
                      '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', 
                      '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', 
                      '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', 
                      '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', 
                      '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', 
                      '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', 
                      '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', 
                      '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', 
                      '54': 'WV', '55': 'WI', '56': 'WY', '72': 'PR'}
SF52 = {v:k for k,v in STATE_FIPS_DICT_52.items()}

time: 857 µs (started: 2022-01-11 08:48:46 -05:00)


In [ ]:
# WORKAROUND FOR NOW: skip field nointernetper for OKLAHOMA

In [64]:
data = []
for sf, abbrv in STATE_FIPS_DICT_52.items():
    df = pd.read_csv(f'es2020/es_{abbrv}.csv')
    
    hh = int(df.num_household.sum())
    hh2020 = int(df.hh2020.sum())
    hu = int(df.num_housingunit.sum())
    hu2020 = int(df.hu2020.sum())
    
    parcel = df.parcelNumAddr.sum()
    resi = df.parcelNumResi.sum()
    agri = df.parcelNumAgri.sum()
    commer = df.parcelNumCommer.sum()
    
    
    infra = df.parcelNumInfra.sum()
    es_infra = df.Health.sum() + df["Public Admin"].sum() + df.Education.sum()
    
    data.append([sf, abbrv, hh, hh2020, hu, hu2020,
                 parcel, resi, agri, commer, infra, es_infra
                ])


/home/nhat/anaconda3/envs/py39/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


time: 2min 7s (started: 2022-01-06 21:39:19 -05:00)


In [74]:
df[['abbrv',
    'parcel','resi', 'agri', 'commer', 'infra']]

,abbrv,parcel,resi,agri,commer,infra
0,AL,2498114,959765,61235,95057,5856
1,AK,268208,125851,2650,6111,698
2,AZ,2830142,2062706,7692,58445,14605
3,AR,1603892,925333,155837,73009,103
4,CA,11056123,7346249,194589,307938,61160
5,CO,2250572,1481746,44243,58907,9794
6,CT,1162425,642443,1594,14327,616
7,DE,402527,347907,4842,14840,1888
8,DC,131793,111610,0,6161,1673
9,FL,9705762,6763035,190949,209264,124505


time: 7.65 ms (started: 2022-01-07 14:54:49 -05:00)


In [65]:
df = pd.DataFrame(data, columns=['sf','abbrv','hh','hh2020', 'hu', 'hu2020', 
                                 'parcel','resi', 'agri', 'commer', 'infra', 'es_infra'])
df['diff%-hh-hu'] = round(abs(df.hu - df.hh) / (df.hu + df.hh) * 200)
df['diff%-hu-parcel'] = round(abs(df.hu - df.parcel) / (df.hu + df.parcel) * 200)
df['diff%-hh-parcel'] = round(abs(df.hh - df.parcel) / (df.hh + df.parcel) * 200)
# df['diff%-hh-resi'] = round(abs(df.hh - df.resi) / (df.hh + df.resi) * 200)
# df['diff%-resi-parcel'] = round(abs(df.resi - df.parcel) / (df.resi + df.parcel) * 200)
df

,sf,abbrv,hh,hh2020,hu,hu2020,parcel,resi,agri,commer,infra,es_infra,diff%-hh-hu,diff%-hu-parcel,diff%-hh-parcel
0,01,AL,1910949,2002629,2346929,2302611,2498114,959765,61235,95057,5856,"3,083.00",20.00,6.00,27.00
1,02,AK,255192,271166,334465,321356,268208,125851,2650,6111,698,750.00,27.00,22.00,5.00
2,04,AZ,2695088,2614924,3186149,3120291,2830142,2062706,7692,58445,14605,"4,683.00",17.00,12.00,5.00
3,05,AR,1185043,1224230,1419687,1401037,1603892,925333,155837,73009,103,"2,105.00",18.00,12.00,30.00
4,06,CA,13298823,13230527,14655385,14383328,11056123,7346249,194589,307938,61160,"28,441.00",10.00,28.00,18.00
5,08,CO,2227739,2174955,2498728,2432800,2250572,1481746,44243,58907,9794,"4,099.00",11.00,10.00,1.00
6,09,CT,1383420,1410239,1538094,1530092,1162425,642443,1594,14327,616,"2,765.00",11.00,28.00,17.00
7,10,DE,373491,378328,468622,449515,402527,347907,4842,14840,1888,635.00,23.00,15.00,7.00
8,11,DC,291789,295987,325112,327459,131793,111610,0,6161,1673,810.00,11.00,85.00,76.00
9,12,FL,8078585,8101436,10028389,9814149,9705762,6763035,190949,209264,124505,"12,582.00",22.00,3.00,18.00


time: 25.4 ms (started: 2022-01-06 21:41:27 -05:00)


In [66]:
df.describe().loc[['mean', '50%','std', 'min', 'max']].astype(int).style.format('{:,}')

,hh,hh2020,hu,hu2020,parcel,resi,agri,commer,infra,es_infra,diff%-hh-hu,diff%-hu-parcel,diff%-hh-parcel
mean,"2,389,206","2,424,479","2,754,271","2,733,937","2,466,254","1,460,522","81,779","78,688","15,025","4,187",15,17,16
50%,"1,712,115","1,731,672","1,934,701","1,923,902","1,581,778","877,021","49,797","54,402","3,536","2,832",15,13,11
std,"2,580,088","2,590,917","2,926,229","2,888,603","2,578,278","1,681,290","105,987","93,477","25,828","4,819",5,17,15
min,"233,518","244,787","287,853","281,935","131,793",0,0,0,0,599,10,1,1
max,"13,298,823","13,230,527","14,655,385","14,383,328","11,616,699","7,346,249","605,683","490,969","124,505","28,441",31,85,76


time: 24.9 ms (started: 2022-01-06 21:41:27 -05:00)


In [ ]:
Within each census block, hh&parcel are most similar (smallest diff%)
=> # passing = average of hh and parcel?


#### Residential passings = housing units (> households)
#### Agri passings = parcelNumAgri
#### Commer passings = parcelNumCommer
#### Infra passings = parcelNumInfra

# Calculate # passings

In [185]:
resi = ['Residential']
commer = ['Commercial', 'Multi']
infra = ['Government' ,'Educational' ,'Religious' ,'Institutional']
valid_passing_cols = resi + commer + infra
es_cols = ['numPassingTotalNAD', 'numPassingValidNAD', 'numPassingResiNAD', 'numPassingCommerNAD', 
            'numPassingInfraNAD', 'numPassingRemNAD', 'addressesNAD']

def addressing_cb(r):
    d = {}
    d['numPassingResiNAD'] = r['Addr_Type'].isin(resi).sum()
    d['numPassingCommerNAD'] = r['Addr_Type'].isin(commer).sum()
    d['numPassingInfraNAD'] = r['Addr_Type'].isin(infra).sum()
    d['numPassingRemNAD'] = (~r['Addr_Type'].isin(valid_passing_cols)).sum()
    
    d['numPassingValidNAD'] = (d['numPassingResiNAD'] 
                               + d['numPassingCommerNAD'] 
                               + d['numPassingInfraNAD'])
    d['numPassingTotalNAD'] =  d['numPassingValidNAD'] + d['numPassingRemNAD']
    
    # numPassingTotalNAD = sum of all (numPassingResiNAD, numPassingCommerNAD, numPassingInfraNAD, numPassingRemNAD)
    # numPassingInfraNAD (better term: community anchor institution)
    
    # combine addresses that belong to same census block
    d['addressesNAD'] = '^&'.join(r['address'])
    
    return pd.Series(d, index = es_cols)


time: 1.03 ms (started: 2022-01-14 11:31:40 -05:00)


In [186]:
state_abbrv = 'dc'
STATE_ABBRV = state_abbrv.upper()
statefips = SF52[STATE_ABBRV]

nad_layer_crs = 'epsg:3857'

with jsonlines.open(f'nad6/nad_{STATE_ABBRV}.jsonl') as reader: 
    state_df = gp.GeoDataFrame.from_features(reader, crs= nad_layer_crs)
    
    # READING TIGER SHP
    tiger_url = "https://www2.census.gov/geo/tiger/TIGER2019/TABBLOCK/tl_2019_{}_tabblock10.zip".format(statefips) 
    print(f'Reading Tiger shp {statefips=} {STATE_FIPS_DICT_52[statefips]=}')
    state_cb = gp.read_file(tiger_url)
    state_cb = state_cb[['GEOID10', 'geometry']]
    state_cb = state_cb.rename({"GEOID10": "GEOID"}, axis = 1)
    print(f"{state_cb.shape=}")
    
    # CONVERT crs: take a bit of time
    state_df = state_df.to_crs(state_cb.crs)
    
    # Point-in-polygon (PIP) spatial join: long time
    state_df = gp.sjoin(state_df, state_cb, how='left', predicate='within')
    # DISOWNED PARCELS: parcels with (lat, lon) not found within state boundaries i.e. parcels w/o GEOID!
    disowned_parcels = state_df['GEOID'].isnull().sum()
    print(f"Number of disowned parcels: {disowned_parcels=}, {(disowned_parcels/state_df.shape[0] * 100) :.5f}%")
    
    # AGGR addressesNAD fields
    state_df['coords'] = 'lon=' + state_df.geometry.x.astype(str) + ';lat=' + state_df.geometry.y.astype(str)
    # concat address fields
    state_df['address'] = state_df[['NatGrid_Coord', 'addr', 'Addr_Type', 'coords']].agg('$%'.join, axis=1)
    # Census block grouping and agg
    state_df = state_df.groupby('GEOID').apply(addressing_cb).reset_index()

Reading Tiger shp statefips='11' STATE_FIPS_DICT_52[statefips]='DC'
state_cb.shape=(6507, 2)
Number of disowned parcels: disowned_parcels=7, 0.00169%
time: 1min (started: 2022-01-14 11:31:41 -05:00)


In [188]:
# PROCESS the df to UPLOAD TO ES!
es_df = state_cb.merge(state_df, how='left', on="GEOID")
es_df = pd.DataFrame(es_df.drop(columns='geometry'))
# del state_cb
# del state_df
for col in es_cols:
    if col == 'addressesNAD':
        es_df[col] = es_df[col].fillna("").astype(str)
    else:
        es_df[col] = es_df[col].fillna(0).astype(int)

time: 15.3 ms (started: 2022-01-14 11:38:11 -05:00)


In [190]:
# save to file
es_df.to_json(f'processed/nad_{STATE_ABBRV}.json', orient='records')

time: 187 ms (started: 2022-01-14 11:38:20 -05:00)


## ES update and upload

In [ ]:
def update_es_mapping(es, index_name):
    # UPDATE ES maaping: add new field
    # null_value i.e. set 0 as default value for the field -> allow indexing and searching https://www.elastic.co/guide/en/elasticsearch/reference/current/null-value.html
    update_request_body = {
        "properties":{
                "numPassingTotalNAD" :       {  "type": "integer",  "null_value": 0 },
                "numPassingValidNAD" :       {  "type": "integer",  "null_value": 0 },
                "numPassingResiNAD" :        {  "type": "integer",  "null_value": 0 },
                "numPassingCommerNAD" :      {  "type": "integer",  "null_value": 0 },
                "numPassingInfraNAD" :       {  "type": "integer",  "null_value": 0 },
                "numPassingRemNAD" :         {  "type": "integer",  "null_value": 0 },
            # https://stackoverflow.com/questions/41862255/null-value-not-permitted-for-text-field
                "addressesNAD" :             {  "type": "text"},
        }
    }
    es.indices.put_mapping(index = index_name, body = update_request_body)
    print('updated index ', index_name)

def upload_es_df(df, index_name):
    idCol = 'GEOID'
    # updating_dict: a dict of all the data to be updated
    updating_dict = df.to_json(orient = "records")
    updating_dict = json.loads(updating_dict)

    # create an actiondf for all actions
    actiondf = df[[idCol]].copy()
    actiondf['_op_type'] = 'update'
    actiondf['_index'] = index_name
    actiondf['_id'] = actiondf[idCol]
    actiondf['doc_as_upsert'] = True
    actiondf['doc'] = updating_dict 
    actiondf.drop(idCol, axis = 1, inplace = True)

    udf = actiondf.to_json(orient = 'records')
    # all actions
    action_list = json.loads(udf) 
    print(f"Start uploading {len(df)} records to {index_name}")
    helpers.bulk(es, action_list)
    print(f"Completed uploading {len(df)} records to {index_name}")

In [ ]:
index_name = f'bossdata{statefips}'
## RUN ONCE
# update_es_mapping(es, index_name)
# upload_es_df(es_df, index_name) 


## Compare NAD w/ Regrid buildings

In [200]:
# 
state_abbrv = 'dc'
STATE_ABBRV = state_abbrv.upper()
statefips = SF52[STATE_ABBRV]

nad_layer_crs = 'epsg:3857'

with jsonlines.open(f'nad6/nad_{STATE_ABBRV}.jsonl') as reader: 
    df = gp.GeoDataFrame.from_features(reader, crs= nad_layer_crs)

time: 13.6 s (started: 2022-01-14 17:14:35 -05:00)


In [201]:
df.head()

,geometry,addr,GUID,NatGrid_Coord,Addr_Type
0,POINT (-8566459.103 4702142.557),37TH Street Southeast 2017 ADDRESS 102 5671 ...,{69CDAFF3-93C1-4529-BA3F-31A6B7B19D4F},18S UJ 30485 03446,Residential
1,POINT (-8566459.103 4702142.557),37TH Street Southeast 2017 ADDRESS 201 5671 ...,{8E4BB74B-58D2-438E-AECC-2F2AD70C2D32},18S UJ 30485 03446,Residential
2,POINT (-8566459.103 4702142.557),37TH Street Southeast 2017 ADDRESS 202 5671 ...,{A36B163B-575F-4B4F-B29A-C14E5720094C},18S UJ 30485 03446,Residential
3,POINT (-8566459.103 4702142.557),37TH Street Southeast 2017 ADDRESS 301 5671 ...,{46C23554-DA95-4E11-8402-C61A6467A915},18S UJ 30485 03446,Residential
4,POINT (-8566459.103 4702142.557),37TH Street Southeast 2017 ADDRESS 302 5671 ...,{BB7D5848-F525-4B0C-A383-8C2754E82E5C},18S UJ 30485 03446,Residential


time: 7.45 ms (started: 2022-01-14 17:14:51 -05:00)


In [ ]:
# HERE NOW: confirm len of addr unique
# NatGrid_Coord ~ 3-to-1 mapping to NAD record

In [202]:
df.shape[0], len(df.addr.unique()), len(df.GUID.unique()), len(df.NatGrid_Coord.unique()), 

(413098, 410411, 413098, 147859)

time: 217 ms (started: 2022-01-14 17:14:54 -05:00)


In [198]:
df.Addr_Type.unique()

array(['Residential', 'Multi', 'Unknown'], dtype=object)

time: 81.2 ms (started: 2022-01-14 11:42:01 -05:00)


### Regrid building